# This is a notebook for querying the Argo instance for the 1.0.1 Beacon release.

-   You can run each cell individually by pressing "shift + enter".
-   For more information, questions, bugs, please contact us on Slack:

    -   https://join.slack.com/t/beacontechnic-wwa5548/shared_invite/zt-2dp1vv56r-tj_KFac0sAKNuAgUKPPDRg.

-   For more information about querying Beacon, please refer to the documentation:
    -   https://maris-development.github.io/beacon/docs/1.0.0/query-docs.html


#### In order to get access to the Beacon endpoint, you need to fill in your unique personal token between the " " in the cell below.


In [ ]:
Token = " "

#### Install the following packages, if you have not already installed them in your environment:

-   pip install requests
-   pip install xarray
-   pip install ipywidgets
-   pip install cartopy
-   pip install h5netcdf
-   pip install netcdf4
-   pip install scipy
-   pip install packaging


#### Import the required packages


In [ ]:
import requests
import json
import xarray as xr
import datetime
import pandas as pd
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

### Beacon Status Check


In [ ]:
response = requests.get(
    "https://beacon-argo.maris.nl/api/health",
    headers={"Authorization": f"Bearer {Token}"},
)
if response.status_code == 200:
    print("Beacon is up and running.")
else:
    print("Beacon is down or not reachable.")
    exit()

#### Retrieve the available columns from the Argo endpoint

Swagger page here: https://beacon-argo.maris.nl/swagger/


In [ ]:
responseinfo = requests.get(
    "https://beacon-argo.maris.nl/api/query/available-columns",
    headers={"Authorization": f"Bearer {Token}"},
)
params = responseinfo.json()

#### Below you can search through the available columns by entering text between the brackets of search_columns(" ").


In [ ]:
def search_columns(search_term):
    search_term = search_term.lower()
    matches = [col for col in params if search_term in col.lower()]

    if matches:
        print("Matching columns:")
        for match in matches:
            print(match)
    else:
        print("No matching columns found.")


search_columns("psal")  # Enter your search term here

#### You can define here your input parameters


In [ ]:
parameter = "PSAL"  # column name
mindate = "2010-01-01"  # yyyy-mm-dd
maxdate = "2011-01-01"  # yyyy-mm-dd
minlon = -180
maxlon = 180
minlat = -90
maxlat = 90
mindepth = 0
maxdepth = 10

#### This will create the query body based on your input parameters, you can add other "query_parameters" and "filters" to suit your needs.

-   For more query examples and explanations, you can take a look at https://maris-development.github.io/beacon/.


In [ ]:
def query(
    parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat, mindepth, maxdepth
):
    body = {
        "query_parameters": [
            {"column_name": parameter, "alias": parameter, "skip_fill_values": True},
            {"column_name": f"{parameter}.units", "alias": "Unit"},
            {"column_name": "cf_datetime", "alias": "datetime"},
            {"column_name": "PRES", "alias": "DEPTH"},
            {"column_name": "LONGITUDE", "alias": "LONGITUDE"},
            {"column_name": "LATITUDE", "alias": "LATITUDE"},
        ],
        "filters": [
            {
                "for_query_parameter": "datetime",
                "min": f"{mindate}T00:00:00",
                "max": f"{maxdate}T00:00:00"
            },
            {"for_query_parameter": "DEPTH", "min": mindepth, "max": maxdepth},
            {"for_query_parameter": "LONGITUDE", "min": minlon, "max": maxlon},
            {"for_query_parameter": "LATITUDE", "min": minlat, "max": maxlat},
        ],
        "output": {"format": "parquet"},
    }
    return body


query_body = query(
    parameter, mindate, maxdate, minlon, maxlon, minlat, maxlat, mindepth, maxdepth
)

#### This is the post request that is sent to Beacon with the above specified body.


In [ ]:
response = requests.post(
    "https://beacon-argo.maris.nl/api/query",
    json.dumps(query_body),
    headers={"Authorization": f"Bearer {Token}", "Content-type": "application/json"},
)

if response.status_code == 204:
    print(
        "No data has been found for your query, please update your input fields above and run the notebook again."
    )
elif response.status_code != 200:
    print(response.text)

In [ ]:
regionname = f"[{minlat},{minlon}],[{maxlat},{maxlon}]"

if not os.path.exists("./Beacon_V1.0.0_Output"):
    os.makedirs("Beacon_V1.0.0_Output")

open(
    f"./Beacon_V1.0.0_Output/Argo_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].nc",
    "wb",
).write(response.content)
df = xr.open_dataset(
    f"./Beacon_V1.0.0_Output/Argo_{parameter}_{regionname}_{mindate}-{maxdate}_[{mindepth}-{maxdepth}m].nc"
).to_dataframe()
df = (
    df.assign(datetime=pd.to_datetime(df["datetime"]))
    .set_index("datetime")
    .sort_index()
)
df

#### Optionally apply a filter on your parameter for quick removal of outliers. Note that this can also be achieved within your Beacon request by applying a filter on your parameter.


In [ ]:
highbound = 40
lowbound = 20
df = df.loc[(df[f"{parameter}"] < highbound) & (df[f"{parameter}"] > lowbound)]
df

#### Plotting of results


In [ ]:
fig = plt.figure(figsize=(23, 18))
ax = plt.axes(projection=ccrs.PlateCarree())

ax.set_extent(
    [-180, 180, -90, 90], crs=ccrs.PlateCarree()
)  # (west, east, south, north)

ax.coastlines(resolution="10m")
ax.gridlines(draw_labels=True)

sc = ax.scatter(
    df["LONGITUDE"],
    df["LATITUDE"],
    c=df[parameter],
    cmap="viridis",
    s=15,
    transform=ccrs.PlateCarree(),
)

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.RIVERS)
ax.add_feature(cfeature.BORDERS)
ax.add_feature(cfeature.LAKES, alpha=0.1)

cbar = plt.colorbar(sc, ax=ax, orientation="vertical", shrink=0.6, label="Value Set 1")
cbar.set_label(f"{parameter} [{df["Unit"].iloc[0]}]")

plt.title(f"{parameter} {regionname} {mindate}-{maxdate} [{mindepth}-{maxdepth}m]")

plt.show()